In [2]:
import numpy as np
from pyspark.sql import SparkSession

In [4]:
# Criando uma sessão spark
spark = SparkSession.builder.appName("EDA_SISU").getOrCreate()

23/11/27 08:57:12 WARN Utils: Your hostname, daniel-VJFE43F11X-XXXXXX resolves to a loopback address: 127.0.1.1; using 192.168.0.157 instead (on interface wlo1)
23/11/27 08:57:12 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/11/27 08:57:13 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [13]:
# Lendo o arquivo csv
dados = spark.read.csv("../data/chamada_regular_sisu_2022_1.csv", header=True, sep="|", encoding="latin1")

In [14]:
# Observando o estado inicial dos dados
dados.show(5)

+----+------+-----+---------------+----------+--------------------+---------+------+-------------+--------------------+---------+----------------+------------+--------------------+-----------+-------+----------------+-------+---------------------+------------------+---------------------+----------------+------+-------+-------+------+------+-------------+--------------+--------------+-------------+-------------+------------+-------------+--------------+--------------------+----+---------------+------------+-------------------+-----+------+-------+-------+------+------+---------------+----------------+----------------+---------------+---------------+--------------+----------+-------------+--------+--------------+
| ANO|EDICAO|ETAPA|       DS_ETAPA|CODIGO_IES|            NOME_IES|SIGLA_IES|UF_IES|CODIGO_CAMPUS|         NOME_CAMPUS|UF_CAMPUS|MUNICIPIO_CAMPUS|CODIGO_CURSO|          NOME_CURSO|       GRAU|  TURNO|DS_PERIODICIDADE|TP_COTA|TIPO_MOD_CONCORRENCIA|  MOD_CONCORRENCIA|QT_VAGAS_CONC

In [15]:
# Permitindo usar SQL nos dados
dados.createOrReplaceTempView("dados")

A partir dos dados iniciais do SISU, vamos fazer uma análise exploratória dos dados para responder as seguintes perguntas: 
- Qual a porcentagem de aprovados?
- Qual a maior e menor nota de aprovação?
- Dos aprovados, quantos porcento foram aceitos na primeira opção? E na segunda?
- Qual a probabilidade de ser aprovado na segunda opção dado que foi reprovado na primeira?
- Qual a porcentagem de cotistas aplicantes?
- Destes, qual a porcentagem de aprovados?
- Qual a porcentagem de cotistas aprovados na primeira opção? E na segunda?
- Qual a probabilidade de ser aprovado na primeira opção sendo um cotista? E na segunda?
- Há diferença na probabilidade de ser aprovado em um curso dado as modalidades de concorrência (cotas, ampla concorrência, etc)?
- Há alguma diferença na probabilidade de ser aprovado em um curso dado o seu turno?
- Há alguma diferença na probabilidade de ser aprovado em um curso dado o seu grau?
- No geral, qual a probabilidade de ser aprovado dada diferentes notas de corte?

**Brincar com as notas e os pesos para aprovação.**

# Qual a taxa de aprovação do ENEM?
Obs: A taxa é considerando primeira e segunda escolha.

In [26]:
# Calculando a taxa de aprovados
qtd_aprovados = spark.sql("SELECT ROUND(COUNT(*), 2) \
                           FROM dados \
                           WHERE APROVADO = 'S'").collect()[0][0]

# Calculando o total de registros
total_de_registros = spark.sql("SELECT COUNT(*) FROM dados").collect()[0][0]

print(f"Taxa de aprovados: {round(qtd_aprovados / total_de_registros * 100, 2)}%")

Taxa de aprovados: 10.18%


# Qual a maior e a menor nota de aprovação?

In [29]:
# Buscando a menor nota de candidatos aprovados
menor_nota = spark.sql("SELECT MIN(NOTA_CANDIDATO) \
                        FROM dados \
                        WHERE APROVADO = 'S'").collect()[0][0]

print(f"Menor nota de candidatos aprovados: {menor_nota}")

Menor nota de candidatos aprovados: 208,79


In [30]:
# Buscando a maior nota de candidatos aprovados
maior_nota = spark.sql("SELECT MAX(NOTA_CANDIDATO) \
                        FROM dados \
                        WHERE APROVADO = 'S'").collect()[0][0]

print(f"Maior nota de candidatos aprovados: {maior_nota}")

Maior nota de candidatos aprovados: 953,14


In [32]:
# Verificando o terceiro quartil das notas de candidatos aprovados
terceiro_quartil = spark.sql("SELECT PERCENTILE(NOTA_CANDIDATO, 0.75) \
                              FROM dados \
                              WHERE APROVADO = 'S'").collect()[0][0]

print(f"75% das notas dos aprovados está abaixo de {terceiro_quartil}")

75% das notas dos aprovados está abaixo de 686.0


In [33]:
# Verificando o primeiro quartil das notas dos candidatos aprovados
primeiro_quartil = spark.sql("SELECT PERCENTILE(NOTA_CANDIDATO, 0.25) \
                              FROM dados \
                              WHERE APROVADO = 'S'").collect()[0][0]

print(f"25% das notas dos aprovados está abaixo de {primeiro_quartil}")


25% das notas dos aprovados está abaixo de 577.0


# Dos aprovados, quantos porcento foram aceitos na primeira opção? E na segunda?

In [34]:
# Buscando o número de aprovados na primeira opção
aprovados_primeira_opcao = spark.sql("SELECT COUNT(*) \
                                      FROM dados \
                                      WHERE APROVADO = 'S' \
                                      AND OPCAO = 1").collect()[0][0]

print(f"Do total de aprovados, {round(aprovados_primeira_opcao / qtd_aprovados * 100, 2)}% foram aceitos na primeira opção")

Do total de aprovados, 63.85% foram aceitos na primeira opção


In [35]:
# Calculando os aprovados somente na segunda opção
aprovados_segunda_opcao = spark.sql("SELECT COUNT(*) \
                                     FROM dados \
                                     WHERE APROVADO = 'S' \
                                     AND OPCAO = 2").collect()[0][0]

print(f"Do total de aprovados, {round(aprovados_segunda_opcao / qtd_aprovados * 100, 2)}% foram aceitos na segunda opção")

Do total de aprovados, 36.15% foram aceitos na segunda opção


# Qual a probabilidade de ser aprovado na segunda opção dado que foi reprovado na primeira?

In [12]:
# Buscando todos os reprovados na primeira opção de curso
reprovados_1_opcao = dados.query("APROVADO == 'N' and OPCAO == 1")

# Buscando todos os aprovados na segunda opção de curso
aprovados_2_opcao = dados.query("APROVADO == 'S' and OPCAO == 2")


In [14]:
# Obtendo a lista de inscritos aprovados e seus respectivos CPFs
df_inscritos_aprovados = aprovados_2_opcao.groupby("CPF").INSCRITO.value_counts().to_frame().reset_index()
df_inscritos_aprovados = df_inscritos_aprovados.drop(columns = "count")

# Obtendo a lista de inscritos reprovados e seus respectivos CPFs
df_inscritos_reprovados = reprovados_1_opcao.groupby("CPF").INSCRITO.value_counts().to_frame().reset_index()
df_inscritos_reprovados = df_inscritos_reprovados.drop(columns = "count")

#qtd_reprovados_opcao_1_aprovados_opcao_2 = 

In [17]:
# Obtendo a quantidade de inscritos aprovados
qtd_reprovados_opcao_1_aprovados_opcao_2 = 0
for inscrito, cpf in zip(df_inscritos_aprovados.INSCRITO, df_inscritos_aprovados.CPF):
    qtd = df_inscritos_reprovados[(df_inscritos_reprovados['CPF'] == cpf) & (df_inscritos_reprovados['INSCRITO'] == inscrito)].shape[0]
    if qtd > 0:
        qtd_reprovados_opcao_1_aprovados_opcao_2 += 1
    else:
        continue

KeyboardInterrupt: 

In [9]:

# Calculando a porcentagem de reprovados na primeira opção de curso
prcnt_reprovados_1_opcao = round(((reprovados_1_opcao.shape[0]/dados.shape[0]) * 100), 2)

# Calculando a porcentagem de aprovados na segunda opção de curso
prcnt_aprovados_2_opcao = round(((aprovados_2_opcao.shape[0]/dados.shape[0]) * 100), 2)

# Calculando a porcentagem de reprovados na primeira opção de curso que foram aprovados na segunda opção de curso
prcnt_reprovados_1_opcao_aprovados_2_opcao = round(((qtd_reprovados_opcao_1_aprovados_opcao_2/dados.shape[0]) * 100), 2)